<a href="https://colab.research.google.com/github/jweir136/Digits1/blob/master/Deep_Titanic_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

np.random.seed(42)

In [0]:
data = pd.read_csv("train.csv")

In [0]:
data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

In [0]:
data['Sex'] = pd.get_dummies(data['Sex'])
data['Embarked'] = pd.get_dummies(data['Embarked'])

In [81]:
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22.0,1,0,7.2500,0
1,1,1,1,38.0,1,0,71.2833,1
2,1,3,1,26.0,0,0,7.9250,0
3,1,1,1,35.0,1,0,53.1000,0
4,0,3,0,35.0,0,0,8.0500,0


In [82]:
data.isnull().any()

Survived    False
Pclass      False
Sex         False
Age          True
SibSp       False
Parch       False
Fare        False
Embarked    False
dtype: bool

In [0]:
data.fillna(value=data['Age'].mean(), inplace=True)

In [0]:
dataX = data.drop(['Survived'], axis=1).values
dataY = data['Survived'].values

In [0]:
scaler = StandardScaler()
dataX = scaler.fit_transform(dataX)


In [0]:
trainX, testX, trainY, testY = train_test_split(dataX, dataY, test_size=.2)

In [0]:
X = tf.placeholder(tf.float32, shape=(None, 7))
y = tf.placeholder(tf.int64, shape=(None))

In [0]:
with tf.name_scope("nn"):
  input_layer = tf.layers.dense(X, 7, tf.nn.relu)
  hidden1 = tf.layers.dense(input_layer, 300, tf.nn.relu)
  hidden2 = tf.layers.dense(hidden1, 100, tf.nn.relu)
  output = tf.layers.dense(hidden2, 2, tf.nn.relu)

In [0]:
with tf.name_scope("loss"):
  cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=output)
  loss = tf.reduce_mean(cross_entropy)

In [0]:
with tf.name_scope("train"):
  optimizer = tf.train.GradientDescentOptimizer(0.01)
  train_op = optimizer.minimize(loss)

In [0]:
with tf.name_scope("eval"):
  correct = tf.nn.in_top_k(output, y, 1)
  accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [0]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [93]:
with tf.Session() as sess:
  init.run()
  
  for i in range(1, 10001):
    sess.run(train_op, feed_dict={X:trainX, y:trainY.reshape(-1, 1)})
    if i % 1000 == 0:
      saver.save(sess, "/tmp/model.ckpt")
      l = loss.eval(feed_dict={X:trainX, y:trainY.reshape(-1, 1)})
      train_acc = accuracy.eval(feed_dict={X:trainX, y:trainY})
      test_acc = accuracy.eval(feed_dict={X:testX, y:testY})
      print(i // 1000, l)

1 0.5218076
2 0.5218076
3 0.5218076
4 0.5218076
5 0.5218076
6 0.5218076
7 0.5218076
8 0.5218076
9 0.5218076
10 0.5218076


In [94]:
train_acc, test_acc

(1.0, 1.0)